In [ ]:
"""
Positives include:

The image is able to be uploaded

Good grid that moves with the zoom in/out to ensure the
image isn't warped

Buttons that ensure the colors that are selected by the 
user vary by a good deal and are easily distinguishable 
from each other

Lots of different brush options
Web-based


Negatives include:

Previous annotations are cleared when the page is refreshed to
change the color or size of the selected tool

Image upload quality is poor

User has to keep changing port number as preoccupation is frequent

Annotations are saved on a labeled .png grid with only the annotations 
visible and a white framing around the image
"""

In [1]:
#required packages: dash, jupyter_dash, plotly, scikit-image
import base64
import io
import json
import plotly.graph_objects as go
import dash
from jupyter_dash import JupyterDash
from dash import dcc, html, Input, Output, State
from skimage import io as skio
import sqlite3
# ChatGPT was attempting to use sqlite to save the previous annotations

app = JupyterDash(__name__, title="Image Annotation")

img = None
fig = go.Figure()

#create buttons for the annotation tools
config = {
    "displaylogo": False,
    "displayModeBar": True,
    "modeBarButtonsToAdd": [
        "drawline",
        "drawopenpath",
        "drawclosedpath",
        "drawcircle",
        "drawrect",
        "eraseshape",
    ]
}

app.layout = html.Div(
    [
        html.H3("Manual image annotation demo"),
        html.Div(
            [
                dcc.Upload(
                    id="upload-image",
                    children=html.Button("Upload Image"),
                    style={"display": "inline-block"},
                ),
            ],
            className="upload-container",
        ),
        dcc.Graph(id="figure_state", config=config),
        
        #create buttons of varying tool colors
        html.Button("Green", id="draw-green-button", n_clicks=0),
        html.Button("Blue", id="draw-blue-button", n_clicks=0),
        html.Button("Red", id="draw-red-button", n_clicks=0),
        html.Button("Purple", id="draw-purple-button", n_clicks=0),
        html.Div(
            [
                html.Label("Tool Size"),
                dcc.Slider(
                    id="line-width-slider",
                    min=1,
                    max=25,
                    step=1,
                    value=1,
                    marks={i: str(i) for i in range(1, 26)},
                ),
            ]
        ),
        html.Div(id="annotation-data", style={"display": "none"}),
    ]
)


def parse_image(contents):
    content_type, content_string = contents.split(",")
    decoded = base64.b64decode(content_string)
    image = skio.imread(io.BytesIO(decoded))
    return image


#ChatGPT attempting to save the annotation data so it doesn't disappear
def store_annotation_data(annotation_data):
    conn = sqlite3.connect('annotations.db')
    cursor = conn.cursor()

    cursor.execute('''CREATE TABLE IF NOT EXISTS annotations
                      (data TEXT)''')

    cursor.execute('DELETE FROM annotations')

    if annotation_data is not None:
        cursor.execute('INSERT INTO annotations (data) VALUES (?)', (annotation_data,))

        conn.commit()
        conn.close()

        return [], annotation_data
    else:
        conn.close()
        return [], []


@app.callback(
    [Output("figure_state", "figure"),
     Output("annotation-data", "children")],

    [Input("upload-image", "contents"),
     Input("line-width-slider", "value"),
     Input("draw-green-button", "n_clicks"),
     Input("draw-blue-button", "n_clicks"),
     Input("draw-red-button", "n_clicks"),
     Input("draw-purple-button", "n_clicks")],

    [State("figure_state", "figure"),
     State("line-width-slider", "value"),
     State("annotation-data", "children"),
     State("figure_state", "relayoutData")], 
)
def update_image_and_layout(
        content,
        line_width,
        green_clicks,
        blue_clicks,
        red_clicks,
        purple_clicks,
        figure,
        slider_value,
        annotation_data,
        relayout_data
):
    ctx = dash.callback_context

    #establishing initial tool color
    line_color = "black"

    if ctx.triggered:
        button_id = ctx.triggered[0]["prop_id"].split(".")[0]

        if button_id == "draw-green-button":
            line_color = "green"
        elif button_id == "draw-blue-button":
            line_color = "blue"
        elif button_id == "draw-red-button":
            line_color = "red"
        elif button_id == "draw-purple-button":
            line_color = "purple"
        else:
            line_color = "black"

    fig = figure if figure else go.Figure()

    if content is not None:
        img = parse_image(content)
        fig = go.Figure(go.Image(z=img))
    else:
        fig = figure if figure else go.Figure()

    fig.update_layout(
        dragmode="drawline",
        newshape=dict(line_color=line_color, line_width=line_width),
        shapes=json.loads(annotation_data) if annotation_data else []
    )

    #ChatGPT trying to save annotation data, cont.
    store_annotation_data(annotation_data)

    return fig, annotation_data


if __name__ == "__main__":
    app.run_server(port=8090, debug=True)
    #may have to change the above port if an error occurs, 
    #to ports such as port=8080, port=8070


Dash is running on http://127.0.0.1:8090/

Dash app running on http://127.0.0.1:8090/


In [ ]:
"""
Positives include:

All annotations can be built upon each other, 
regardless of changes to color and size of the tool

Negatives include:

Extremely lengthy image upload time if at all

No grid

No save button

No different brushes

Zoom in/out buttons don't work

Python app can be a little hefty to run
"""

In [ ]:
#required packages: tkinter, pillow
from tkinter import *
from tkinter.colorchooser import askcolor
from tkinter import filedialog
from PIL import ImageTk, Image, ImageGrab

class Image_Markup(object):
    
    #creating the initial setup

    DEFAULT_COLOR = 'black'
    ZOOM_STEP = 0.1

    def __init__(self):
        #establishing the buttons needed to activate each feature
        self.root = Tk()
        self.root.title('SEE Image Markup Tool')

        self.zoom_factor = 1.0
        self.pan_start_x = 0
        self.pan_start_y = 0
        self.original_image = None

        self.brush_button = Button(self.root, text='Brush', command=self.use_brush)
        self.brush_button.grid(row=0, column=1)

        self.color_button = Button(self.root, text='Color', command=self.choose_color)
        self.color_button.grid(row=0, column=2)

        self.eraser_button = Button(self.root, text='Eraser', command=self.use_eraser)
        self.eraser_button.grid(row=0, column=3)

        self.browse_button = Button(self.root, text='Browse', command=self.browseFiles)
        self.browse_button.grid(row=0, column=4)

        self.clear_button = Button(self.root, text='Clear All', command=self.clear_all)
        self.clear_button.grid(row=0, column=5)

        self.choose_size_button = Scale(self.root, from_=10, to=50, orient=HORIZONTAL)
        self.choose_size_button.grid(row=0, column=6)

        self.save_button = Button(self.root, text='Save', command=self.save_canvas)
        self.save_button.grid(row=0, column=9)

        self.zoom_in_button = Button(self.root, text='+', command=self.zoom_in)
        self.zoom_in_button.grid(row=0, column=7)

        self.zoom_out_button = Button(self.root, text='-', command=self.zoom_out)
        self.zoom_out_button.grid(row=0, column=8)

        self.pan_button = Button(self.root, text='Pan', command=self.activate_pan)
        self.pan_button.grid(row=0, column=9)

        self.c_frame = Frame(self.root)
        self.c_frame.grid(row=1, columnspan=11)

        self.c = Canvas(self.c_frame, bg='white')
        self.c.grid(row=0, column=0, sticky='nsew')
        self.c.bind('<Configure>', lambda event: self.root.after_idle(self.calculate_zoom_factor))  # Bind configure event

        self.setup()
        #I wanted the user to be able to zoom in and out using keyboard shortcuts
        #In later iterations of the code I tried to make it so that the program would check
        #to see the operating system and adjust to command- or control- accordingly
        self.root.bind('<Command-plus>', self.zoom_in)
        self.root.bind('<Command-minus>', self.zoom_out)
        self.root.mainloop()

    def setup(self):
        self.old_x = None
        self.old_y = None
        self.line_width = self.choose_size_button.get()
        self.color = self.DEFAULT_COLOR
        self.eraser_on = False
        self.active_button = self.brush_button
        self.zoom_in_factor = 1.1
        self.zoom_out_factor = 0.9
        self.c.bind('<B1-Motion>', self.paint)
        self.c.bind('<ButtonRelease-1>', self.reset)

    def use_brush(self):
        self.activate_button(self.brush_button)

    def choose_color(self):
        self.eraser_on = False
        self.color = askcolor(color=self.color)[1]

    def use_eraser(self):
        self.activate_button(self.eraser_button, eraser_mode=True)

    def clear_all(self):
        self.c.delete("all")

    def activate_button(self, some_button, eraser_mode=False):
        self.active_button.config(relief=RAISED)
        some_button.config(relief=SUNKEN)
        self.active_button = some_button
        self.eraser_on = eraser_mode

    def paint(self, event=None):
        self.line_width = self.choose_size_button.get()
        paint_color = 'white' if self.eraser_on else self.color
        if self.old_x and self.old_y:
            self.c.create_line(self.old_x, self.old_y, event.x, event.y,
                               width=self.line_width, fill=paint_color,
                               capstyle=ROUND, smooth=TRUE, splinesteps=36)
        self.old_x = event.x
        self.old_y = event.y

    def reset(self, event=None):
        self.old_x, self.old_y = None, None

    def save_canvas(self):
        x = self.root.winfo_rootx() + self.c.winfo_x()
        y = self.root.winfo_rooty() + self.c.winfo_y()
        x1 = x + self.c.winfo_width()
        y1 = y + self.c.winfo_height()
        print('x, y, x1, y1:', x, y, x1, y1)
        
    def zoom_in(self):
        self.zoom_factor += self.ZOOM_STEP
        self.update_canvas()

    def zoom_out(self):
        if self.zoom_factor > self.ZOOM_STEP:
            self.zoom_factor -= self.ZOOM_STEP
            self.update_canvas()

    def update_canvas_frame(self, event):
        self.c.configure(scrollregion=self.c.bbox('all'))

    def activate_pan(self):
        self.c.bind('<ButtonPress-1', self.start_pan)
        self.c.bind('<B1-Motion>', self.pan)
        self.c.bind('<ButtonRelease-1', self.end_pan)

    def start_pan(self, event):
        self.pan_start_x = event.x
        self.pan_start_y = event.y

    def pan(self, event):
        dx = event.x - self.pan_start_x
        dy = event.y - self.pan_start_y
        self.c.scan_dragto(dx, dy, gain=1)

    def end_pan(self, event):
        self.c.unbind('<ButtonPress-1>')
        self.c.unbind('<B1-Motion>')
        self.c.unbind('<ButtonRelease-1>')

    def handle_configure(self, event):
        self.root.after(1, self.calculate_zoom_factor)

    def update_canvas(self, event=None):
        if self.original_image is None:
            return

        width = int(self.original_image.width * self.zoom_factor)
        height = int(self.original_image.height * self.zoom_factor)
        self.displayed_image = self.original_image.resize((width, height))

        self.c.config(width=width, height=height)
        self.c.delete('all')

        if self.displayed_image:
            img_tk = ImageTk.PhotoImage(self.displayed_image)
            self.c.create_image(0, 0, anchor=NW, image=img_tk)
            self.c.image = img_tk

    def browseFiles(self):
        filename = filedialog.askopenfilename(initialdir="/", title="Select a File",
                                              filetypes=(("png files", "*.png"),
                                                         ("jpg files", "*.jpg"),
                                                         ("jpeg files", "*.jpeg")))
        if filename:
            self.original_image = Image.open(filename)
            self.update_canvas()

    def calculate_zoom_factor(self):
        canvas_width = self.c.winfo_width()
        canvas_height = self.c.winfo_height()

        if canvas_width > 0 and canvas_height > 0 and self.original_image:
            image_width, image_height = self.original_image.size

            width_ratio = canvas_width / image_width
            height_ratio = canvas_height / image_height

            self.zoom_factor = min(width_ratio, height_ratio)

            self.update_canvas()

Image_Markup()

2023-06-19 16:02:37.918 python[63019:1990324] +[CATransaction synchronize] called within transaction


In [ ]:
"""
Positives include:

Image uploades relatively quickly

Simple buttons

No save button

Can layer annotations with changes in color


Negatives include:

Uploaded image type limited to .jpg files

No options to change size or type of annotation tool

When zoomed in/out, all annotations are erased

No save button

Annotations can be drawn over the edge of the canvas
"""

In [4]:
#required packages: tkinter, pillow
from tkinter import *
from tkinter.colorchooser import askcolor
from tkinter import filedialog
from PIL import ImageTk, Image, ImageGrab


class ImageMarkupTool:
    DEFAULT_COLOR = 'black'
    ZOOM_STEP = 0.1

    def __init__(self):
        self.root = Tk()
        self.root.title('SEE Image Markup Tool')

        self.zoom_factor = 1.0
        self.pan_start_x = 0
        self.pan_start_y = 0
        self.original_image = None

        self.toolbar = Frame(self.root)
        self.toolbar.pack(side=TOP, fill=X)

        self.browse_button = Button(self.toolbar, text='Browse', command=self.browse_files)
        self.browse_button.pack(side=LEFT, padx=5, pady=5)

        self.zoom_in_button = Button(self.toolbar, text='+', command=self.zoom_in)
        self.zoom_in_button.pack(side=LEFT, padx=5, pady=5)

        self.zoom_out_button = Button(self.toolbar, text='-', command=self.zoom_out)
        self.zoom_out_button.pack(side=LEFT, padx=5, pady=5)

        self.pan_button = Button(self.toolbar, text='Pan', command=self.activate_pan)
        self.pan_button.pack(side=LEFT, padx=5, pady=5)

        self.brush_button = Button(self.toolbar, text='Brush', command=self.activate_brush)
        self.brush_button.pack(side=LEFT, padx=5, pady=5)

        self.color_button = Button(self.toolbar, text='Color', command=self.choose_color)
        self.color_button.pack(side=LEFT, padx=5, pady=5)

        self.eraser_button = Button(self.toolbar, text='Eraser', command=self.activate_eraser)
        self.eraser_button.pack(side=LEFT, padx=5, pady=5)

        self.canvas = Canvas(self.root, bg='white')
        self.canvas.pack(fill=BOTH, expand=True)
        self.canvas.bind('<Configure>', self.handle_configure)

        self.setup()
        self.root.mainloop()

    def setup(self):
        self.old_x = None
        self.old_y = None
        self.line_width = 10
        self.color = self.DEFAULT_COLOR
        self.eraser_on = False
        self.active_button = None
        self.zoom_in_factor = 1.1
        self.zoom_out_factor = 0.9
        self.canvas.bind('<B1-Motion>', self.paint)
        self.canvas.bind('<ButtonRelease-1>', self.reset)

    def activate_button(self, button, eraser_mode=False):
        if self.active_button:
            self.active_button.config(relief=RAISED)
        button.config(relief=SUNKEN)
        self.active_button = button
        self.eraser_on = eraser_mode

    def activate_brush(self):
        self.activate_button(self.brush_button, eraser_mode=False)

    def choose_color(self):
        color = askcolor(color=self.color)[1]
        if color:
            self.color = color

    def activate_eraser(self):
        self.activate_button(self.eraser_button, eraser_mode=True)

    def paint(self, event):
        if self.active_button != self.brush_button and self.active_button != self.eraser_button:
            return

        self.line_width = 10
        paint_color = 'white' if self.eraser_on else self.color

        if self.old_x and self.old_y:
            self.canvas.create_line(self.old_x, self.old_y, event.x, event.y, width=self.line_width, fill=paint_color, capstyle=ROUND, smooth=TRUE, splinesteps=36)

        self.old_x = event.x
        self.old_y = event.y

    def reset(self, event):
        self.old_x, self.old_y = None, None

    def zoom_in(self):
        self.zoom_factor += self.ZOOM_STEP
        self.update_canvas()

    def zoom_out(self):
        if self.zoom_factor - self.ZOOM_STEP >= self.ZOOM_STEP:
            self.zoom_factor -= self.ZOOM_STEP
            self.update_canvas()

    def activate_pan(self):
        self.canvas.bind('<ButtonPress-1>', self.start_pan)
        self.canvas.bind('<B1-Motion>', self.pan)
        self.canvas.bind('<ButtonRelease-1>', self.end_pan)

    def start_pan(self, event):
        self.pan_start_x = event.x
        self.pan_start_y = event.y

    def pan(self, event):
        dx = event.x - self.pan_start_x
        dy = event.y - self.pan_start_y
        self.canvas.scan_dragto(dx, dy, gain=1)

    def end_pan(self, event):
        self.canvas.unbind('<ButtonPress-1>')
        self.canvas.unbind('<B1-Motion>')
        self.canvas.unbind('<ButtonRelease-1>')

    def handle_configure(self, event):
        self.root.after(1, self.calculate_zoom_factor)

    def update_canvas(self, event=None):
        if self.original_image is None:
            return
    
        width = int(self.original_image.width * self.zoom_factor)
        height = int(self.original_image.height * self.zoom_factor)
        displayed_image = self.original_image.resize((width, height))

        self.canvas.config(width=width, height=height)
        self.canvas.delete('all')

        if displayed_image:
            img_tk = ImageTk.PhotoImage(displayed_image)
            self.canvas.create_image(0, 0, anchor=NW, image=img_tk)
            self.canvas.image = img_tk

    def browse_files(self):
        filename = filedialog.askopenfilename(initialdir="/", title="Select a File",
                                              filetypes=(("PNG files", "*.png"),
                                                         ("JPEG files", "*.jpg;*.jpeg")))
        if filename:
            self.original_image = Image.open(filename)
            self.zoom_factor = 1.0
            self.update_canvas()

    def calculate_zoom_factor(self):
        if self.original_image is None:
            return

        canvas_width = self.canvas.winfo_width()
        canvas_height = self.canvas.winfo_height()
        image_width, image_height = self.original_image.size
        width_ratio = canvas_width / image_width
        height_ratio = canvas_height / image_height
        self.zoom_factor = min(width_ratio, height_ratio)
        self.update_canvas()


if __name__ == '__main__':
    tool = ImageMarkupTool()


2023-06-19 15:55:01.491 python[62322:1966611] +[CATransaction synchronize] called within transaction
